# Домашняя работа

In [2]:
import pandas as pd

## Задача 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [27]:
data = pd.read_csv('ml-latest-small/ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [28]:
def lifetime(data):
    return data.timestamp.max() - data.timestamp.min()

In [33]:
?pd.DataFrame

In [55]:
lifetime_series = data.groupby('userId').apply(lifetime)
lifetime_data = pd.DataFrame(data = lifetime_series, columns = ['lifetime']).reset_index()
lifetime_data.head()

,userId,lifetime
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


In [30]:
rating_data = data.groupby('userId').count().reset_index()[['userId','rating']]
many_ratings= rating_data[rating_data.rating > 100]
many_ratings.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [41]:
joined_data = pd.merge(many_ratings, lifetime_data, on = 'userId', how = 'left').sort_values('rating', ascending = False)
joined_data.head()

,userId,rating,lifetime
208,547,2391,501810535
217,564,1868,1516358
243,624,1735,457492507
2,15,1700,471393496
23,73,1610,220585493


In [44]:
average_lifetime = joined_data.lifetime.mean()
# значение в секундах
average_lifetime 

40080507.4496124

In [46]:
# значение в днях
average_lifetime/(3600*24)

463.8947621482917

## Задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:
 - таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
 - аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [4]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [5]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [6]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [51]:
pd.merge(rzd, auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer')


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [54]:
pd.merge(rzd, auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer').merge(client_base, on = 'client_id', how = 'left')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задача 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

### Ответ:
Координаты необходимо записывать в отдельную таблицу, которая будет состоять из столбца с широтой, столбца с долготой и столбца времени. Связь с таблицей визитов будет осуществляться по времени визита, а связь с таблицей покупок по времени покупки (если времени покупки в таблице покупок нет, то его необходимо добавить). 

Из набора координат можно дополнительно получить следующую информацию:
1. В каких местах пользователь бывает чаще всего - на основе этой информации можно выдавать рекламу товаров для конкретного пользователя (например, пользователь часто заходит в приложение из фитнес центра, тогда ему можно предлагать рекламу спортивных товаров)
2. Полный набор координат для одного пользователя дает картину перемещений данного пользователя, по которой также можно делать выводы об его интересах с целью предложения соотвествующих товаров и услуг (например, если пользователь перемещается на довольно большие расстояния, осещает разные города и страны, то ему будет интересна туристическая реклама)